# **Install YoloV8(pip install)**

In [ ]:
!pip install ultralytics

from IPython.display import clear_output, Image
clear_output()
!yolo checks

Ultralytics YOLOv8.0.58 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


# **Import Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!7z x /content/drive/MyDrive/Colab\ Notebooks/Data\ Science/Learning/PyTorch/archive.zip

In [ ]:
import pandas as pd

dataset = pd.read_csv('/content/data/train_solution_bounding_boxes (1).csv')
dataset.head()

In [ ]:
dataset.isnull().sum()

In [ ]:
from PIL import Image
import os 

def normalize(labels, image_dir):
  for idx in labels.index:
    name = labels['image'][idx]
    image = os.path.join(image_dir, name)
    if os.path.isfile(image):
      img = Image.open(image)
      width, height = img.size
      labels['width'][idx] = labels['width'][idx] / width
      labels['height'][idx] = labels['height'][idx] / height
      labels['xc'][idx] = labels['xc'][idx] / width
      labels['yc'][idx] = labels['yc'][idx] / height


In [ ]:
dataset['width'] = dataset.loc[:, 'xmax'] - dataset.loc[:, 'xmin']
dataset['height'] = dataset.loc[:, 'ymax'] - dataset.loc[:, 'ymin']
dataset['xc'] = dataset.loc[:, 'xmin'] + dataset.loc[:, 'width']/2
dataset['yc'] = dataset.loc[:, 'ymin'] + dataset.loc[:, 'height']/2
normalize(dataset, '/content/data/training_images')
dataset.head()

In [ ]:
os.rename('/content/data/testing_images', '/content/data/test')
os.rename('/content/data/training_images', '/content/data/train')
file = open('/content/data/data.yaml', 'w')
file.writelines(
    'train: ../train/images' + '\n' +
    'val: ../val/images' + '\n' +
    'test: ../test/images' + '\n' +
    'nc: 1' + '\n' +
    "names: ['car']" + '\n'
)
file.close()

In [ ]:
from sklearn.model_selection import train_test_split

images = dataset.loc[:]
images.drop_duplicates('image', keep = 'first', inplace = True)
X_train, X_valid, _, _ = train_test_split(images, images['xmin'], test_size = 0.2, random_state = 42)
X_train.head()

In [ ]:
training_path = '/content/data/train'
valid_path = '/content/data/val'
os.mkdir(training_path + '/images')
os.mkdir(training_path + '/labels')
os.mkdir(valid_path)
os.mkdir(valid_path + '/images')
os.mkdir(valid_path + '/labels')

def move_images(dir_src, dir_dst, images):
  for i in images.index:
    image = images['image'][i]
    image_path = os.path.join(dir_src, image)
    os.rename(image_path, os.path.join(dir_dst, f'images/{image}'))

def add_label(dir, images):
 for i in images.index:
   image = os.path.join(dir, images['image'][i])
   if os.path.isfile(image):
     label = image.replace('/images', '/labels').replace('.jpg', '.txt').replace('.png', '.txt')
     if os.path.isfile(label):
       with open(label, 'a') as f:
        f.write('\n' + '0' + ' ' + str(images['xc'][i]) + ' ' + str(images['yc'][i]) + ' ' + str(images['width'][i]) + ' ' + str(images['height'][i]))
     else:
      with open(label, 'a') as f:
        f.write('0' + ' ' + str(images['xc'][i]) + ' ' + str(images['yc'][i]) + ' ' + str(images['width'][i]) + ' ' + str(images['height'][i]))

In [ ]:
move_images(training_path, training_path, X_train)
move_images(training_path, valid_path, X_valid)
add_label(os.path.join(training_path, 'images'), dataset)
add_label(os.path.join(valid_path, 'images'), dataset)

In [ ]:
import glob
import numpy as np

files = glob.glob('/content/data/train/vid_*.jpg')
num_of_images = len(files)
if num_of_images > 0:
  num = int(num_of_images * 5 / 100)
  shuffle = np.random.permutation(num_of_images)
  '''for i in shuffle[: num]:
    image = files[i].split('/')[-1]
    os.rename(files[i], f'/content/data/train/images/{image}')
    label = image.replace('.jpg', '.txt').replace('.png', '.txt')
    with open(os.path.join('/content/data/train/labels', label), 'w+') as f:
      f.close()'''
  for idx in shuffle:
    os.remove(files[idx])

# **Train Model**

In [ ]:
from ultralytics import YOLO

def set_res_dir(train = True):
  res_dir_count = len(glob.glob('/content/runs/detect/results_*'))
  if train:
    res_dir = f'results_{res_dir_count + 1}'
  else:
    res_dir = f'results_{res_dir_count}'
  return res_dir

def train(model = 'yolov8n.pt', data = 'coco128.yaml', epochs = 25):
  res_dir = set_res_dir()
  model = YOLO(model = model)
  model.train(data = data, epochs = epochs, name = res_dir, imgsz = 640)
  return f'/content/runs/detect/{res_dir}/results.png', f'/content/runs/detect/{res_dir}/weights/best.pt'

In [ ]:
models = [
    'yolov8n.pt',
    'yolov8s.pt',
    'yolov8m.pt',
    'yolov8l.pt',
    'yolov8x.pt'
]
data = '/content/data/data.yaml'
sources = r'/content/data/test/images/*.[jp][pn]g'
results = []
inferences = []
best_models = []
for model in models:
  result, best_model = train(model, data, 30)
  results.append(result)
  best_models.append(best_model)

# **Inference**

In [ ]:
!wget https://app.roboflow.com/ds/s4gd3l9oss?key=pxE4Cs4MS4
!7z x /content/s4gd3l9oss?key=pxE4Cs4MS4

In [ ]:
!pip install torchmetrics[detection]

from torchmetrics.detection import mean_ap

def meanAveragePrecision(predict, target):
  metric = mean_ap.MeanAveragePrecision(box_format = 'cxcywh')
  metric.update(predict, target)
  result = metric.compute()
  mAp = result['map']
  return mAp.numpy()

In [ ]:
import torch
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

def get_bboxes(labels):
  bboxes = []
  with open(labels, 'r') as f:
    lines = f.readlines()
    for line in lines:
      values = line.split(' ')
      if len(values) < 4: break
      boxes = []
      boxes.append(float(values[1]))
      boxes.append(float(values[2])) 
      boxes.append(float(values[3]))
      boxes.append(float(values[4].split('\n')[0]))
      bboxes.append(boxes)
  return bboxes

def desnormalize(bboxes, image):
  image = Image.open(image).convert('RGB')
  width, height = image.size
  for bbox in bboxes:
    xmin = bbox[0] - bbox[2]/2
    ymin = bbox[1] - bbox[3]/2
    xmax = bbox[0] + bbox[2]/2
    ymax = bbox[1] + bbox[3]/2
    bbox[0] = xmin * width
    bbox[1] = ymin * height
    bbox[2] = xmax * width
    bbox[3] = ymax * height
  return bboxes

def inference(model_name, source):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  value = model_name.split('/')[-3].split('_')[-1]
  files = glob.glob(source + '/*.[jp][pn]g')
  results = []
  targets = []
  mAps = []
  for file in files:
    dir = file.replace('/images', '/labels').replace('.jpg', '.txt').replace('.png', '.txt')
    target = {}
    result = {}
    bboxes = get_bboxes(dir)
    target['boxes'] = torch.tensor(bboxes).to(device)
    target['labels'] = torch.zeros(len(target['boxes']), dtype = torch.int64).to(device)
    targets.append(target)
    image = file.split('/')[-1]
    inference_dir = f'inference_{value}/{image}'
    model = YOLO(model_name)
    output = model.predict(file, name = inference_dir, save = True, save_txt = True)
    result['boxes'] = output[0].boxes.xywhn.to(device)
    result['labels'] = torch.zeros(len(output[0].boxes), dtype = torch.int64).to(device)
    result['scores'] = output[0].boxes.cls.to(device)
    results.append(result)
    img_result = []
    img_target = []
    img_result.append(result)
    img_target.append(target)
    mAp = meanAveragePrecision(img_result, img_target)
    map_file = f'/content/runs/detect/{inference_dir}/map.txt'
    with open(map_file, 'a') as f:
      f.writelines(
          f'Image: {image}\n mAp: {mAp}\n Boxes: ' + '{\n'
      )
      for box in target['boxes']:
        f.writelines(
            ' ' + str(box[0]) + ' ' + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + '\n'
        )
      f.writelines('}')
    mAps.append(mAp)
  return results, targets, mAps

def show_image(image, bboxes, save = False, dir = None):
  bboxes = get_bboxes(bboxes)
  bboxes = desnormalize(bboxes, image)
  img = np.array(Image.open(image).convert('RGB')).astype('float')
  for bbox in bboxes:
    img = cv2.rectangle(
        img,
        (int(bbox[0]), int(bbox[1])),
        (int(bbox[2]), int(bbox[3])),
        color = (255, 0, 0),
        thickness = 2
    )
  if save:
    if dir is None:
      dir = '/content/show_image'
      if not os.path.isdir(dir):
          os.mkdir(dir)
    name = image.split('/')[-1]
    cv2.imwrite(os.path.join(dir, name), img)
  else:
    cv2.imshow(img)

In [ ]:
def set_ground_truth(images, dir_dst):
  for img in glob.glob(f'{images}/*[jp][pn]g'):
    label = img.replace('/images', '/labels').replace('.jpg', '.txt').replace('.png', '.txt')
    img_name = img.split('/')[-1]
    dir = f'{dir_dst}/{img_name}/ground_truth'
    os.mkdir(dir)
    show_image(img, label, save = True, dir = dir)

In [ ]:
for img in glob.glob('/content/valid/images/*.[jp][pn]g'):
  dir_label = img.replace('/images', '/labels').replace('.jpg', '.txt').replace('.png', '.txt')
  with open(dir_label, 'r') as f:
    lines = f.readlines()
    if len(lines) == 0:
      os.remove(dir_label)
      os.remove(img)

In [ ]:
results_list = []
targets_list = []
mAps_list = []
source = '/content/valid/images'
for best_model in best_models:
  results, targets, mAps = inference(best_model, source)
  value = best_model.split('/')[-3].split('_')[-1]
  dir_dst = f'/content/runs/detect/inference_{value}'
  set_ground_truth(source, dir_dst)
  results_list.append(results)
  targets_list.append(targets)
  mAps_list.append(mAps)

In [ ]:
from ultralytics import YOLO
source = '/content/valid/images'
results, targets, maps = inference('/content/last.pt', source)